In [1]:

from __future__ import print_function

import sys

# sys.path.extend(['/home/jogi/git/repository/dcnn_acdc'])
sys.path.extend(['/home/jorg/repository/dcnn_acdc'])

import matplotlib
import matplotlib.pyplot as plt

from matplotlib import cm


from datetime import datetime
from pytz import timezone
import itertools
import copy

import SimpleITK as sitk
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init

import numpy as np
import dill
import os
import glob
from collections import OrderedDict

from in_out.load_data import ACDC2017DataSet
from config.config import config
from utils.experiment import Experiment, ExperimentHandler
from utils.batch_handlers import TwoDimBatchHandler
from utils.test_handler import ACDC2017TestHandler, test_ensemble
from utils.test_results import TestResults, load_all_results
from utils.referral_handler import ReferralHandler, ReferralResults
from plotting.learning_curves import loss_plot
from plotting.uncertainty_plots import analyze_slices
from plotting.main_seg_results import plot_seg_erros_uncertainties
from plotting.result_plots import plot_referral_results

from models.dilated_cnn import BaseDilated2DCNN

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Constants aka fixed variables
ROOT_DIR = "/home/jorg/repository/dcnn_acdc"
LOG_DIR = os.path.join(ROOT_DIR, "logs")
CYCLE_OFFSET = 100000
# set seeds
SEED = 4325
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.enabled = True
np.random.seed(SEED)

# IMPORTANT - THE FOLD WE'RE USING FOR TESTING. NOTE: Training and testing fold should be the same
# we raise an error in the test_ensemble method if this is not the case!
TEST_FOLD = 3

# baseline: 
exp_base = {3: "20180509_18_36_23_dcnn_f3_150KE_lr2e02",
            2: "20180509_18_36_28_dcnn_f2_150KE_lr2e02",
            1: "20180509_18_36_32_dcnn_f1_150KE_lr2e02",
            0: "20180330_09_56_39_dcnnv1_150000E_lr2e02"}
exp_base_path = os.path.join(LOG_DIR, exp_base[TEST_FOLD])


exp_mc005_brier = {3: "20180428_09_11_35_dcnn_mc_f3p005_brier_150KE_lr2e02",
                   2: "20180426_14_47_23_dcnn_mc_f2p005_brier_150KE_lr2e02",
                   1: "20180426_14_45_50_dcnn_mc_f1p005_brier_150KE_lr2e02",
                   0: "20180418_15_02_36_dcnn_mcv1_150000E_lr2e02"}
exp_mc005_brier_path = os.path.join(LOG_DIR, exp_mc005_brier[TEST_FOLD])


# 20180508_11_47_05_dcnn_mc_f2p01_brier_20KE_lr2e02
# 20180426_14_14_39_dcnn_mc_f2p01_brier_150KE_lr2e02
exp_mc01_brier = {3: "20180426_14_14_57_dcnn_mc_f3p01_brier_150KE_lr2e02",
                  2: "20180426_14_14_39_dcnn_mc_f2p01_brier_150KE_lr2e02",
                  1: "20180426_14_13_46_dcnn_mc_f1p01_brier_150KE_lr2e02",
                  0: "20180418_15_02_05_dcnn_mcv1_150000E_lr2e02"}
exp_mc01_brier_path = os.path.join(LOG_DIR, exp_mc01_brier[TEST_FOLD])

print("!!! IMPORTANT !!! - EVALUATING MODELS ON HOLD OUT IMAGES IN FOLD {}".format(TEST_FOLD))

exper_base = ExperimentHandler.load_experiment(exp_base_path)
exper_hdl_base = ExperimentHandler(exper_base, use_logfile=False)
exper_hdl_base.set_root_dir(ROOT_DIR)
exper_hdl_base.set_model_name("Base DCNN fold-{} (p={})".format(
                                     exper_hdl_base.exper.run_args.fold_ids[0],
                                     exper_hdl_base.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_base.exper.model_name))


exper_mc005_brier = ExperimentHandler.load_experiment(exp_mc005_brier_path)
exper_hdl_mc005_brier = ExperimentHandler(exper_mc005_brier, use_logfile=False)
exper_hdl_mc005_brier.set_root_dir(ROOT_DIR)
exper_hdl_mc005_brier.set_model_name("MC-dropout-brier fold-{} (p={})".format(
                                     exper_hdl_mc005_brier.exper.run_args.fold_ids[0],
                                     exper_hdl_mc005_brier.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc005_brier.exper.model_name))


exper_mc01_brier = ExperimentHandler.load_experiment(exp_mc01_brier_path)
exper_hdl_mc01_brier = ExperimentHandler(exper_mc01_brier, use_logfile=False)
exper_hdl_mc01_brier.set_root_dir(ROOT_DIR)
exper_hdl_mc01_brier.set_model_name("MC-dropout brier fold-{} (p={})".format(
                                exper_hdl_mc01_brier.exper.run_args.fold_ids[0],
                                exper_hdl_mc01_brier.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc01_brier.exper.model_name))


try:
    del test_set
except:
    pass

!!! IMPORTANT !!! - EVALUATING MODELS ON HOLD OUT IMAGES IN FOLD 3
Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180509_18_36_23_dcnn_f3_150KE_lr2e02/stats/exper_stats.dll
Model name Base DCNN fold-3 (p=0.5)
Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180428_09_11_35_dcnn_mc_f3p005_brier_150KE_lr2e02/stats/exper_stats.dll
Model name MC-dropout-brier fold-3 (p=0.05)
Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180426_14_14_57_dcnn_mc_f3p01_brier_150KE_lr2e02/stats/exper_stats.dll
Model name MC-dropout brier fold-3 (p=0.1)


### Load referral results

In [12]:
exper_dict = exp_mc01_brier
referral_thresholds = [0.08]  # [0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24] 

# ref_results_pos_only = ReferralResults(exper_dict, referral_thresholds, pos_only=True, print_results=False,
#                                       fold=None)
ref_results = ReferralResults(exper_dict, referral_thresholds, print_results=False,
                              fold=3)

WARNING - only loading results for fold 3
INFO - Loading referral results for thresholds [0.08]
WARNING - referral positives-only=False


In [19]:
for disease_cat, org_dice in ref_results.org_dice_per_dcat[0.08].iteritems():
    print(disease_cat)
    print(org_dice[0])
    ref_dice = ref_results.ref_dice_per_dcat[0.08][disease_cat]
    print(ref_dice[0])

NOR
[ 0.99847943  0.84392926  0.91133383  0.92530869]
[ 0.99847943  0.89709172  0.93129461  0.94395577]
DCM
[ 0.99810203  0.83756206  0.87525279  0.95767116]
[ 0.99810203  0.89686559  0.91567948  0.97222092]
MINF
[ 0.99796954  0.79849576  0.84513434  0.92756414]
[ 0.99796954  0.89304431  0.91083464  0.95625084]
ARV
[ 0.99736843  0.86093702  0.8732802   0.89994509]
[ 0.99736843  0.94058354  0.93734362  0.94794664]
HCM
[ 0.99831073  0.82964516  0.89888238  0.87348585]
[ 0.99831073  0.90236302  0.92185173  0.91561592]


In [20]:
from utils.uncertainty_blobs import UncertaintyBlobStats

referral_thresholds = [0.08] #, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24]
ublob = UncertaintyBlobStats(exp_mc01_brier, referral_thresholds)

  0%|          | 0/1 [00:00<?, ?it/s]

Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02/stats/exper_stats.dll
Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180426_14_13_46_dcnn_mc_f1p01_brier_150KE_lr2e02/stats/exper_stats.dll
Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180426_14_14_39_dcnn_mc_f2p01_brier_150KE_lr2e02/stats/exper_stats.dll
Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180426_14_14_57_dcnn_mc_f3p01_brier_150KE_lr2e02/stats/exper_stats.dll


100%|██████████| 1/1 [00:01<00:00,  1.26s/it]

INFO - Saved results to /home/jorg/repository/dcnn_acdc/data/Folds/UncertaintyBlobStats.dll


In [22]:
ublob.min_area_size

{0.08: [486.49842271293375, 364.35331230283913]}

### Generate slice features

In [ ]:
from in_out.generate_slice_features import SliceFeatureGenerator


referral_threshold = 0.14

slice_features = SliceFeatureGenerator(exper_dict, verbose=False, referral_threshold=referral_threshold)
slice_features.generate_features(num_of_bins=25, threshold=None, normalize=False)

### Create dataset based on slice-features to regress referral-improvements per slice

In [ ]:
from in_out.dataset_slice_referral import SliceReferralDataSet

cardiac_phase = 0
slice_dataset = SliceReferralDataSet(ref_results, slice_features, referral_threshold=referral_threshold,
                                     verbose=True)
x_train, y_train, x_test, y_test, train_ids, test_ids =  \
    slice_dataset.split_train_test_set(fold_id=3, cardiac_phase=cardiac_phase)
print(np.unique(train_ids))

### Run regressor on slice feature dataset

In [ ]:
from sklearn.ensemble import RandomForestRegressor as rfreg
from sklearn.metrics import r2_score

cardiac_phase = 1
slice_dataset = SliceReferralDataSet(ref_results, slice_features, referral_threshold=referral_threshold,
                                     verbose=True)
slice_dataset.verbose = False
importances = 0
rf = rfreg(n_estimators=100)
for fold_id in np.arange(4):
    x_train, y_train, x_test, y_test, train_ids, test_ids = \
            slice_dataset.split_train_test_set(fold_id=fold_id, cardiac_phase=cardiac_phase)
    rf.fit(x_train, y_train)
    y_test_pred = rf.predict(x_test)
    importances += rf.feature_importances_
    
print(importances * 1./4)
r2_metric = r2_score(y_test, y_test_pred)
print(r2_metric)